In [1]:
import boto3
import pandas as pd
from io import StringIO


/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
bucket_name = 'clustertraces'
file_key = 'borg_traces_data.csv'

# Creating a boto3 client
s3 = boto3.client('s3')

# Object rtracing
response = s3.get_object(Bucket=bucket_name, Key=file_key)

# Load CSV
csv_content = response['Body'].read().decode('utf-8')

# Dataframe declaration
df = pd.read_csv(StringIO(csv_content))


df.head()

,Unnamed: 0,time,instance_events_type,collection_id,scheduling_class,collection_type,priority,alloc_collection_id,instance_index,machine_id,...,assigned_memory,page_cache_memory,cycles_per_instruction,memory_accesses_per_instruction,sample_rate,cpu_usage_distribution,tail_cpu_usage_distribution,cluster,event,failed
0,0,0,2,94591244395,3,1,200,0,144,168846390496,...,0.014435,0.000415,NaN,NaN,1.0,[0.00314331 0.00381088 0.00401306 0.00415039 0...,[0.00535583 0.00541687 0.00548553 0.00554657 0...,7,FAIL,1
1,1,2517305308183,2,260697606809,2,0,360,221495397286,335,85515092,...,0.000000,0.000000,NaN,NaN,1.0,[1.23977661e-05 1.23977661e-05 1.23977661e-05 ...,[1.23977661e-05 1.23977661e-05 1.23977661e-05 ...,7,FAIL,1
2,2,195684022913,6,276227177776,2,0,103,0,376,169321752432,...,0.010422,0.000235,0.939919,0.001318,1.0,[0.01344299 0.01809692 0.0201416 0.02246094 0...,[0.02902222 0.02929688 0.0295105 0.0296936 0...,7,SCHEDULE,0
3,3,0,2,10507389885,3,0,200,0,1977,178294817221,...,0.041626,0.000225,1.359102,0.007643,1.0,[0.03704834 0.04125977 0.04290771 0.04425049 0...,[0.05535889 0.05584717 0.05633545 0.05718994 0...,8,FAIL,1
4,4,1810627494172,3,25911621841,2,0,0,0,3907,231364893292,...,0.000272,0.000010,NaN,NaN,1.0,[0. 0. 0. 0. 0...,[0.00041485 0.00041485 0.00041485 0.00041485 0...,2,FINISH,0


In [4]:
df.shape

(405894, 34)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405894 entries, 0 to 405893
Data columns (total 34 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Unnamed: 0                       405894 non-null  int64  
 1   time                             405894 non-null  int64  
 2   instance_events_type             405894 non-null  int64  
 3   collection_id                    405894 non-null  int64  
 4   scheduling_class                 405894 non-null  int64  
 5   collection_type                  405894 non-null  int64  
 6   priority                         405894 non-null  int64  
 7   alloc_collection_id              405894 non-null  int64  
 8   instance_index                   405894 non-null  int64  
 9   machine_id                       405894 non-null  int64  
 10  resource_request                 405120 non-null  object 
 11  constraint                       405894 non-null  object 
 12  co

In [6]:
df.isnull().sum()

Unnamed: 0                              0
time                                    0
instance_events_type                    0
collection_id                           0
scheduling_class                        0
collection_type                         0
priority                                0
alloc_collection_id                     0
instance_index                          0
machine_id                              0
resource_request                      774
constraint                              0
collections_events_type                 0
user                                    0
collection_name                         0
collection_logical_name                 0
start_after_collection_ids              0
vertical_scaling                      959
scheduler                             959
start_time                              0
end_time                                0
average_usage                           0
maximum_usage                           0
random_sample_usage               

**Columns like cycles_per_instruction and memory_accesses_per_instruction offer little value when missing in almost all rows. These fields are likely telemetry data, and their near-total absence means the model would learn noise or constant values if we imputed them.**

In [7]:
df.drop(['cycles_per_instruction', 'memory_accesses_per_instruction'], axis=1, inplace=True)

**We impute lower perentage of missing values with median. Median handles skewed distributions better than mean, avoids outlier influence, and preserves distribution shape for classifiers**

In [8]:
df['vertical_scaling'].fillna(df['vertical_scaling'].median(), inplace=True)

**Mode imputation preserves categorical variable structure without introducing new categories.**

In [9]:
df['scheduler'].fillna(df['scheduler'].mode()[0], inplace=True)

In [10]:
df.isnull().sum()

Unnamed: 0                       0
time                             0
instance_events_type             0
collection_id                    0
scheduling_class                 0
collection_type                  0
priority                         0
alloc_collection_id              0
instance_index                   0
machine_id                       0
resource_request               774
constraint                       0
collections_events_type          0
user                             0
collection_name                  0
collection_logical_name          0
start_after_collection_ids       0
vertical_scaling                 0
scheduler                        0
start_time                       0
end_time                         0
average_usage                    0
maximum_usage                    0
random_sample_usage              0
assigned_memory                  0
page_cache_memory                0
sample_rate                      0
cpu_usage_distribution           0
tail_cpu_usage_distr

In [11]:
import ast

def safe_parse(val):
    try:
        return ast.literal_eval(val)
    except:
        return None  # Keep it None if parsing fails

# Apply to full column — DO NOT dropna()
df['resource_request'] = df['resource_request'].apply(safe_parse)

In [12]:
df['resource_request'].head()

0    {'cpus': 0.020660400390625, 'memory': 0.014434...
1    {'cpus': 0.00724029541015625, 'memory': 0.0013...
2    {'cpus': 0.048583984375, 'memory': 0.004165649...
3    {'cpus': 0.0704345703125, 'memory': 0.04162597...
4    {'cpus': 0.00244903564453125, 'memory': 0.0002...
Name: resource_request, dtype: object

In [13]:
# Extract CPU and memory if parsed successfully
df['resource_cpus'] = df['resource_request'].apply(lambda x: x.get('cpus') if isinstance(x, dict) else None)
df['resource_memory'] = df['resource_request'].apply(lambda x: x.get('memory') if isinstance(x, dict) else None)

In [14]:
# Impute the actual missing values
df['resource_cpus'].fillna(df['resource_cpus'].median(), inplace=True)
df['resource_memory'].fillna(df['resource_memory'].median(), inplace=True)

In [15]:
# Drop original
df.drop('resource_request', axis=1, inplace=True)

In [16]:
df.isnull().sum()

Unnamed: 0                     0
time                           0
instance_events_type           0
collection_id                  0
scheduling_class               0
collection_type                0
priority                       0
alloc_collection_id            0
instance_index                 0
machine_id                     0
constraint                     0
collections_events_type        0
user                           0
collection_name                0
collection_logical_name        0
start_after_collection_ids     0
vertical_scaling               0
scheduler                      0
start_time                     0
end_time                       0
average_usage                  0
maximum_usage                  0
random_sample_usage            0
assigned_memory                0
page_cache_memory              0
sample_rate                    0
cpu_usage_distribution         0
tail_cpu_usage_distribution    0
cluster                        0
event                          0
failed    

**Treating Date Time Dataset**

In [17]:
df['start_time'] = pd.to_datetime(df['start_time'], errors='coerce')
df['end_time'] = pd.to_datetime(df['end_time'], errors='coerce')

In [18]:
df['duration_sec'] = (df['end_time'] - df['start_time']).dt.total_seconds()

**One Hot Encoding**

In [19]:
from sklearn.preprocessing import LabelEncoder
label_cols = ['user', 'scheduler', 'cluster', 'scheduling_class', 'collection_type']
for col in label_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

**Data Visualization**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.countplot(x='failed', data=df, palette=['green', 'red'])
plt.xticks([0, 1], ['Unfailed', 'Failed'])
plt.title('Job Failure Distribution')
plt.ylabel('Number of Jobs')
plt.xlabel('Job Status')
plt.show()
